In [1]:
# 주요 라이브러리 import
import warnings
warnings.filterwarnings(action='ignore')
import time
from IPython.display import Image
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import *
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso, ElasticNet

from matplotlib import rc, font_manager
import matplotlib.font_manager as fm
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

# 디렉토리 및 파일 이름에 맞추어 변경
font_location = '../Jalnan.ttf' #윈도우안에있는 폰트파일주소.
# font_location = './NanumGothic.ttf'
font_name = fm.FontProperties(fname=font_location).get_name()
rc('font', family=font_name)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.o

In [2]:
df = pd.read_csv('../../data.csv') # 2019년 여행설문

In [3]:
df[:100]

,ID,Q,QQ,MON_EXP_1,MON_EXP_2,MON_EXP_3,MON_EXP_4,MON_EXP_5,MON_EXP_6,S5,...,국내_여행방문지_기타당일8,국내_여행방문지_기타당일9,국내_여행방문지_기타당일10,국내_여행방문지_기타당일11,국내_여행방문지_기타당일12,국내_여행방문지_기타당일13,국내_여행방문지_기타당일14,국내_여행방문지_기타당일15,국내_여행방문지_기타당일16,국내_여행방문지_기타당일17
0,11010550341_1000000,1,0,1,2,2,2,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11010550341_1000005,1,0,1,2,2,2,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11010550341_1000007,2,0,1,2,2,2,2,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11010550341_1000012,1,0,1,2,2,2,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11010550341_1000013,1,0,1,2,2,2,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,11020580301_1221834,1,0,1,2,2,2,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,11020580301_1221840,1,0,1,2,2,2,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,11020580301_1221847,1,0,1,2,2,2,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,11020580301_1221861,1,0,1,2,2,2,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
##input 년도에 여행한 사람만 추출하기 전 확인
year = 2019 ##년도
df.iloc[50:100,1:15][df['D_TRA1_SYEAR']==year]

,Q,QQ,MON_EXP_1,MON_EXP_2,MON_EXP_3,MON_EXP_4,MON_EXP_5,MON_EXP_6,S5,S5A_1,S5A_2,S5A_3,S5A_4,S5A_5
50,2,0,2,1,1,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN
53,2,0,1,2,2,2,2,1,NaN,NaN,NaN,NaN,NaN,NaN
55,1,0,2,2,1,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN
56,2,0,1,2,1,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN
58,2,0,1,2,1,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN
59,2,0,1,2,1,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN
60,1,0,1,2,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN
61,1,0,1,2,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN
62,1,0,2,2,2,2,1,2,NaN,NaN,NaN,NaN,NaN,NaN
63,1,0,1,2,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df['D_TRA1_SYEAR'].value_counts()

2019.0    27281
2018.0       40
Name: D_TRA1_SYEAR, dtype: int64

In [6]:
##year 년도에 여행한 사람만 추출
df = df[df['D_TRA1_SYEAR']==year] 

In [7]:
## 관광/휴양 이 포함되지 않는 row 제거
df = df[(df['MON_EXP_1']!=0) | (df['MON_EXP_2']!=0.0) | (df['MON_EXP_4']!=0)]

In [8]:
## 추천도, 재방문의사 보통이상만 추출
df = df[(df['A13']>=3) & (df['A14']>=3)]
df

,ID,Q,QQ,MON_EXP_1,MON_EXP_2,MON_EXP_3,MON_EXP_4,MON_EXP_5,MON_EXP_6,S5,...,국내_여행방문지_기타당일8,국내_여행방문지_기타당일9,국내_여행방문지_기타당일10,국내_여행방문지_기타당일11,국내_여행방문지_기타당일12,국내_여행방문지_기타당일13,국내_여행방문지_기타당일14,국내_여행방문지_기타당일15,국내_여행방문지_기타당일16,국내_여행방문지_기타당일17
1,11010550341_1000005,1,0,1,2,2,2,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11010550341_1000007,2,0,1,2,2,2,2,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11010550341_1000012,1,0,1,2,2,2,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11010550341_1000013,1,0,1,2,2,2,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,11010550341_1000018,1,0,1,2,2,2,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48229,39020600401_1319375,1,0,1,2,2,2,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48246,3902061022A_1382196,1,0,1,2,2,2,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48247,3902061022A_1382200,1,0,2,1,2,2,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48260,39020620091_1382242,1,0,1,2,2,2,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
## 1차 필요컬럼
columns_list = [
    'D_TRA1_SYEAR',
    'A3_1',
    'A3_2',
    'A3_3',
    'A3_4',
    'A3_5',
    'A3_6',
    'A3_7',
    'A3_8',
    'A3_9',
    'A3_10',
    'A3_11',
    'A3_12',
    'A3_13',
    'A3_14',
    'A3_15',
    'A3_16',
    'A3_17',
    'A3_18',
    'A3_19',
    'A3_20',
    'A3_21',
    'BSEX',
    'BAGE',
    'BAGE',
    'BMON',
    'BARA',
    'A12',
    'A10',
    'A7A',
    'A7',
    'A7B_1',
    'A7B_2',
    'A7B_3',
    'A7B_4',
    'A7B_5',
    'A7B_6',
    'A7B_7',
    'A2_1',
    'A2_2',
    'A2_3',
    'A5_1',
    'A5_2',
    'A5_3',
    'A6_1',
    'A6_2',
    'A6_3',
    'A6_4',
    'A6_5',
    'A6_6',
    'A6_7',
    'A6_8',
    'A6_9',
    'A6_10',
    'A6_11',
    'A6_12',
    'A6_13',
    'A6_14',
    'D_TRA1_SMONTH',
    'D_TRA2_SMONTH',
    'D_TRA3_SMONTH',
    'D_TRA4_SMONTH',
    'D_TRA5_SMONTH',
    'D_TRA6_SMONTH',
    'D_TRA7_SMONTH',
    'D_TRA8_SMONTH',
    'D_TRA1_1_SPOT1',
    'D_TRA1_1_SPOT2',
    'D_TRA1_1_SPOT3',
    'D_TRA1_1_SPOT4',
    'D_TRA1_1_SPOT5',
    'D_TRA1_1_SPOT6',
    'D_TRA1_1_SPOT7',
    'D_TRA1_1_SPOT8',
    'D_TRA1_1_SPOT9',
    'D_TRA1_1_SPOT10',
    'D_TRA2_1_SPOT1',
    'D_TRA2_1_SPOT2',
    'D_TRA2_1_SPOT3',
    'D_TRA2_1_SPOT4',
    'D_TRA2_1_SPOT5',
    'D_TRA2_1_SPOT6',
    'D_TRA2_1_SPOT7',
    'D_TRA2_1_SPOT8',
    'D_TRA2_1_SPOT9',
    'D_TRA2_1_SPOT10',
    'D_TRA3_1_SPOT1',
    'D_TRA3_1_SPOT2',
    'D_TRA3_1_SPOT3',
    'D_TRA3_1_SPOT4',
    'D_TRA3_1_SPOT5',
    'D_TRA3_1_SPOT6',
    'D_TRA3_1_SPOT7',
    'D_TRA3_1_SPOT8',
    'D_TRA3_1_SPOT9',
    'D_TRA3_1_SPOT10',
    'D_TRA4_1_SPOT1',
    'D_TRA4_1_SPOT2',
    'D_TRA4_1_SPOT3',
    'D_TRA4_1_SPOT4',
    'D_TRA4_1_SPOT5',
    'D_TRA4_1_SPOT6',
    'D_TRA4_1_SPOT7',
    'D_TRA4_1_SPOT8',
    'D_TRA4_1_SPOT9',
    'D_TRA4_1_SPOT10',
    'D_TRA5_1_SPOT1',
    'D_TRA5_1_SPOT2',
    'D_TRA5_1_SPOT3',
    'D_TRA5_1_SPOT4',
    'D_TRA5_1_SPOT5',
    'D_TRA5_1_SPOT6',
    'D_TRA5_1_SPOT7',
    'D_TRA5_1_SPOT8',
    'D_TRA5_1_SPOT9',
    'D_TRA5_1_SPOT10',
    'D_TRA6_1_SPOT1',
    'D_TRA6_1_SPOT2',
    'D_TRA6_1_SPOT3',
    'D_TRA6_1_SPOT4',
    'D_TRA6_1_SPOT5',
    'D_TRA6_1_SPOT6',
    'D_TRA6_1_SPOT7',
    'D_TRA6_1_SPOT8',
    'D_TRA6_1_SPOT9',
    'D_TRA6_1_SPOT10',
    'D_TRA7_1_SPOT1',
    'D_TRA7_1_SPOT2',
    'D_TRA7_1_SPOT3',
    'D_TRA7_1_SPOT4',
    'D_TRA7_1_SPOT5',
    'D_TRA7_1_SPOT6',
    'D_TRA7_1_SPOT7',
    'D_TRA7_1_SPOT8',
    'D_TRA7_1_SPOT9',
    'D_TRA7_1_SPOT10',
    'D_TRA8_1_SPOT1',
    'D_TRA8_1_SPOT2',
    'D_TRA8_1_SPOT3',
    'D_TRA8_1_SPOT4',
    'D_TRA8_1_SPOT5',
    'D_TRA8_1_SPOT6',
    'D_TRA8_1_SPOT7',
    'D_TRA8_1_SPOT8',
    'D_TRA8_1_SPOT9',
    'D_TRA8_1_SPOT10',
    'D_TRA1_S_Day',
    'D_TRA2_S_Day',
    'D_TRA3_S_Day',
    'D_TRA4_S_Day',
    'D_TRA5_S_Day',
    'D_TRA6_S_Day',
    'D_TRA7_S_Day',
    'D_TRA8_S_Day',
]
    
    

In [10]:
df2 = df[columns_list]

In [11]:
df2.reset_index(inplace=True)
del df2['index']
df2

,D_TRA1_SYEAR,A3_1,A3_2,A3_3,A3_4,A3_5,A3_6,A3_7,A3_8,A3_9,...,D_TRA8_1_SPOT9,D_TRA8_1_SPOT10,D_TRA1_S_Day,D_TRA2_S_Day,D_TRA3_S_Day,D_TRA4_S_Day,D_TRA5_S_Day,D_TRA6_S_Day,D_TRA7_S_Day,D_TRA8_S_Day
0,2019.0,1.0,2.0,NaN,NaN,NaN,6.0,NaN,NaN,NaN,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019.0,1.0,2.0,3.0,NaN,NaN,6.0,7.0,8.0,9.0,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019.0,1.0,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019.0,1.0,2.0,3.0,NaN,NaN,6.0,NaN,NaN,NaN,...,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22567,2019.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22568,2019.0,NaN,2.0,NaN,NaN,NaN,6.0,NaN,8.0,9.0,...,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22569,2019.0,NaN,2.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22570,2019.0,NaN,NaN,NaN,NaN,5.0,6.0,NaN,NaN,NaN,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
